Carga de librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import scipy as sci
from sklearn.neural_network import MLPRegressor
import bokeh.plotting as bpl
from bokeh.io import export_png
from bokeh.models import PrintfTickFormatter
import math
import bokeh.layouts as bly

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Definición de funciones necesarias para entrenar el modelo de alcance

In [3]:
def entrenador(arreglo,alcances):
    """Entrena el modelo utilizando un arreglo de publicaciones o un dataframe y sus alcances.
    
    Parameters:
        arreglo (arreglo de numpy, también puede ser un dataframe):
            Arreglo multidimensional con los valores de las métricas para cada publicación.
            Cada publicación está en una fila del arreglo.
            El orden de las métricas debe ser el siguiente [likes,love,angry,wow,haha,sad,shares].
        alcances (arreglo de numpy, tambien puede ser una serie):
            Arreglo unidimensional con los valores de los alcances para cada publicación.
            Cada publicación está en una fila del arreglo.
        
    Returns:
        red (red neuronal de Sklearn):
            Modelo de red neuronal entrenada para predecir los alcances de publicaciones.
            
    """
    logtrain = np.log1p(arreglo)
    logpredi = np.log1p(alcances)
    
    red = MLPRegressor(alpha=0.01, hidden_layer_sizes = (10,), max_iter = 50000, 
                 activation = 'logistic', learning_rate = 'adaptive',solver= 'lbfgs')
    
    red.fit(logtrain,logpredi)
    
    return red

In [4]:
def predictor(arreglo,modelo):
    """Predice los alcances para un arreglo de publicaciones o un dataframe.
    
    Parameters:
        arreglo (arreglo de numpy, también puede ser un dataframe):
            Arreglo multidimensional con los valores de las métricas para cada publicación.
            Cada publicación está en una fila del arreglo.
            El orden de las métricas debe ser el siguiente [likes,love,angry,wow,haha,sad,shares].
            
        modelo (modelo de sklearn):
            El modelo de predicción entrenado previamente
            
    Returns:
        alcances (arreglo de numpy):
            Arreglo con los alcances para cada publicación.
            
    """
    logdata = np.log1p(arreglo)
    predata = modelo.predict(logdata)
    bacdata = np.expm1(predata)
    
    return bacdata

Carga y procesado de datos para entrenar la red para predecir alcance

In [5]:
data = pd.read_csv("../data/originales/posts.csv")

In [6]:
metricas = ['likes', 'love', 'angry', 'wow', 'haha', 'sad', 'shares']

In [7]:
data["reacciones"] = data[metricas].sum(1)

In [8]:
fdata=data[(data["scope"]!=0)&(data["reacciones"]>10)&(data["reacciones"]<=data["scope"])]

In [9]:
mdata = fdata[metricas + ["scope"]]

In [10]:
mdata[:3]

,likes,love,angry,wow,haha,sad,shares,scope
0,18,7,0,0,0,0,4,3660
1,526,117,189,15,51,8,107,77468
2,28,1,0,0,0,0,13,4399


In [11]:
arr_metricas = mdata[metricas].values

In [12]:
arr_metricas

array([[ 18,   7,   0, ...,   0,   0,   4],
       [526, 117, 189, ...,  51,   8, 107],
       [ 28,   1,   0, ...,   0,   0,  13],
       ...,
       [ 23,   0,   1, ...,   0,   3,   1],
       [ 13,   0,   4, ...,   0,   0,   0],
       [ 24,   0,   0, ...,   0,   0,   0]], dtype=int64)

In [13]:
arr_alcances = mdata["scope"].values

Entrenado de la red predictora de alcance

In [14]:
red = entrenador(arr_metricas,arr_alcances)

Carga de datos de publicaciones y temas

In [15]:
datap = pd.read_csv("../data/originales/Posts_CDMX.tsv",sep='\t')
datat = pd.read_csv("../data/originales/Temas_CDMX.tsv",sep='\t')

D:\Apps\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Proceso de datos de temas y publicaciones

In [16]:
datat.columns=[cadena + "_T" for cadena in datat.columns]

In [17]:
datat.rename(columns={"id_T":"idTema"},inplace=True)

Mezcla de los datos de publicaciones y de temas

In [18]:
datamix = pd.merge(datap,datat,how="left",on="idTema")

Llenado de datos vacíos a cero

In [19]:
datafp = datamix[metricas].fillna(0)

Predicción de alcances para todas las publicaciones

In [20]:
datapv = datafp.values

In [21]:
predicciones = predictor(datapv,red)

Conversión a DataFrame e incorporación al frame general

In [22]:
prediccion = pd.DataFrame(predicciones,columns=["Alcance_estimado"],index=datafp.index)

In [23]:
prediccion["Alcance_estimado"] = prediccion["Alcance_estimado"].apply(lambda x: max(x,10))

In [24]:
datamix["Alcance_estimado"] = prediccion

In [25]:
datamix["reacciones"] = datamix[metricas].sum(1)

In [26]:
datamix["estado_T"].value_counts()

Ciudad de México    51320
Name: estado_T, dtype: int64

Definición de Estado a calificar y parámetros para el modelo, alcmax es el alcance máximo posible, es decir la población total a alcanzar
pubmax es el número máximo de publicaciones para fijar el tope de la calificación, usualmente 100 0 200 funciona bien, dependiendo de la ciudad.

In [27]:
estado = ["Ciudad de México"]
ciudad = "CDMX"
alcmax = 85000000
pubmax = 200

In [28]:
p = bpl.figure(plot_width=600,plot_height=600,x_axis_type="log",y_axis_type="log",title="Alcance vs Reacciones, " + ciudad)

In [29]:
p.circle(x=datamix["reacciones"],y=datamix["Alcance_estimado"],alpha=0.1)

GlyphRenderer(id='1038', ...)

In [30]:
p.xaxis.axis_label = 'Suma de Reacciones'
p.yaxis.axis_label = 'Alcance estimado'
p.yaxis[0].formatter = PrintfTickFormatter(format="%5f")
p.xaxis[0].formatter = PrintfTickFormatter(format="%5f")

In [31]:
bpl.show(p)

In [32]:
variable = "Alcance_estimado"
hist = np.histogram(datamix[variable],bins=1000)

In [33]:
np.log10(hist[1])

array([1.7567865 , 4.06323478, 4.36319117, ..., 7.06021762, 7.06065256,
       7.06108707])

In [34]:
p = bpl.figure(title="Distribución de alcances, " + ciudad)

In [35]:
p.line(x=np.log10(hist[1]),y=np.log10(hist[0]))

D:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


GlyphRenderer(id='1124', ...)

In [36]:
bpl.show(p)

In [37]:
export_png(p, filename="../imagenes/procesados/distcal_" + ciudad + ".png")

'E:\\proyectos\\ds\\virality\\imagenes\\procesados\\distcal_CDMX.png'

Filtrado de las publicaciones a aquellas que están en el estado y selección de columnas importantes

In [38]:
data_filt=datamix[(datamix["estado_T"].isin(estado))][metricas+["score_T","Alcance_estimado","reacciones","idTema","nombre_T"]]

Sustitución de los alcances mayores al alcance máximo por el alcance máximo

In [39]:
data_filt["Alcance_est_top"] = data_filt["Alcance_estimado"].apply(lambda x: min(x,alcmax))

Definición de la función para calcular el alcance extra al alcance de la publicación con mayor alcance para cada tema

In [40]:
def alcance_extra(serie,atope):
    nserie = serie.apply(lambda x: min(x,atope))
    serie_s = nserie.sort_values(ascending=False)
    index = serie_s.index
    a_max  = min(atope,serie_s.max())
    r = (atope - a_max)/atope
    rango = pd.Series(pd.RangeIndex(0,len(serie_s)),index=index)
    mults = np.power(r,rango)
    return (serie_s*mults).sum()-a_max

Agrupación de publicaciones por tema

In [41]:
grupos = data_filt.groupby(["idTema","nombre_T"])

Cálculo del alcance extra para cada tema

In [42]:
por_tema = grupos.apply(lambda x: alcance_extra(x["Alcance_est_top"],alcmax)).to_frame("Alcance_extra")

Cálculos de alcances máximos, suma de alcances y número de publicaciones para cada tema

In [43]:
por_tema["Alcance_max_top"] = grupos["Alcance_est_top"].max()
por_tema["Alcance_suma"] = por_tema["Alcance_max_top"] + por_tema["Alcance_extra"]
por_tema["Publicaciones"] = grupos["Alcance_estimado"].size()
por_tema["reacciones"] = grupos["reacciones"].sum()

In [44]:
alcmin = por_tema["Alcance_suma"].min()
alcmax2 = por_tema["Alcance_suma"].max()

In [45]:
alcmax2

26780911.901004456

Definición de la función para escalar los valores y calcular las calificaciones 

In [46]:
def scale(inp_domain,out_range,valor):
    valor_estimado = ((out_range[1]-out_range[0])*(valor-inp_domain[0])/(inp_domain[1]-inp_domain[0])) + out_range[0]
    return np.clip(valor_estimado,out_range[0],out_range[1])

Definición de pesos para las calificaciones, deben de sumar a 100

In [47]:
peso_alcance = 80
peso_temas = 20

Cálculo de las calificaciones por alcance, por temas y total

In [48]:
por_tema["Cal_alcance"] = scale((np.log10(alcmin),np.log10(alcmax)),(0,peso_alcance),np.log10(por_tema["Alcance_suma"]))
por_tema["Cal_publicaciones"] = scale((np.log10(3),np.log10(pubmax)),(0,peso_temas),np.log10(por_tema["Publicaciones"]))
por_tema["Calificacion"] = por_tema["Cal_alcance"] + por_tema["Cal_publicaciones"]

Definición del frame de salida

In [49]:
salida = por_tema[["reacciones","Alcance_max_top","Alcance_extra","Alcance_suma","Publicaciones","Cal_alcance","Cal_publicaciones","Calificacion"]]

Ordenando de mayor a menor por calificación y alcance total

In [50]:
lista_final = salida.sort_values(["Calificacion","Alcance_suma"],ascending=False)

In [51]:
lista_final.sort_values(["Alcance_suma"],ascending=False)

,,reacciones,Alcance_max_top,Alcance_extra,Alcance_suma,Publicaciones,Cal_alcance,Cal_publicaciones,Calificacion
idTema,nombre_T,,,,,,,,
11945,FGJ dio a conocer retrato hablado de mujer que raptó a Fátima,991899.0,8.363861e+06,1.841705e+07,2.678091e+07,46,73.499129,13.001052,86.500181
11897,FGJ cuenta con imágenes de mujer que robó a Fátima; ofrecen 2 mdp por su paradero,418292.0,6.010955e+06,1.730371e+07,2.331466e+07,52,72.718960,13.584913,86.303873
12400,Confirman primer caso de coronavirus en CDMX,469731.0,4.663649e+06,1.609407e+07,2.075772e+07,28,72.065114,10.636900,82.702014
827,Localizan el cuerpo de Norberto Ronquillo en Xochimilco,448824.0,4.417099e+06,1.625416e+07,2.067126e+07,136,72.041620,18.163383,90.205003
12934,Choque de dos trenes en Metro Tacubaya deja como saldo 1 muerto y 41 heridos,385488.0,2.633192e+06,1.750652e+07,2.013971e+07,97,71.894989,16.554013,88.449002
...,...,...,...,...,...,...,...,...,...
1813,Captan supuesto secuestro en la glorieta de Pabellón Bosques (Cuaj),0.0,5.711978e+01,0.000000e+00,5.711978e+01,1,0.000000,0.000000,0.000000
1518,[FM] PRUEBA TURO B,0.0,5.711978e+01,0.000000e+00,5.711978e+01,1,0.000000,0.000000,0.000000
871,repetido12,0.0,5.711978e+01,0.000000e+00,5.711978e+01,1,0.000000,0.000000,0.000000


In [64]:
variable = "Alcance_suma"
hist = np.histogram(lista_final[variable],bins=100)

In [65]:
p = bpl.figure(title="Distribución de calificaciones, " + ciudad,toolbar_location=None)

In [66]:
p.vbar(x=hist[1],top=hist[0],width=(hist[1][1]-hist[1][0])*0.9)

GlyphRenderer(id='1664', ...)

In [67]:
bpl.show(p)

In [68]:
escala = 20
a = 0.7
x = np.linspace(0,100, 99)
y = sci.stats.gamma.cdf(x,a=a,scale=escala)

In [69]:
lim100 = sci.stats.gamma.cdf(100,a=a,scale=escala)
lim100

0.9969537579688733

In [70]:
serie_data = por_tema[variable].sort_values()

In [71]:
percentiles = np.percentile(serie_data,range(1,100))
percentiles = np.append(percentiles,[serie_data.max()])
rang1 = (np.array(range(1,100))/100)
rang1 = np.append(rang1,[lim100])
interp1 = np.interp(serie_data,percentiles,rang1)

In [72]:
frame = serie_data.to_frame().copy()

In [73]:
frame["inter1"] = interp1

In [74]:
frame[-3:]

,,Alcance_suma,inter1
idTema,nombre_T,,
12400,Confirman primer caso de coronavirus en CDMX,2.075772e+07,0.995056
11897,FGJ cuenta con imágenes de mujer que robó a Fátima; ofrecen 2 mdp por su paradero,2.331466e+07,0.995862
11945,FGJ dio a conocer retrato hablado de mujer que raptó a Fátima,2.678091e+07,0.996954


In [75]:
p = bpl.figure(plot_width=400,plot_height=400,title="Distribución acumulativa empírica",toolbar_location=None)

In [76]:
p.circle(x=serie_data,y=interp1,color="red",radius=50000,alpha=0.1)
p.line(x=percentiles,y=rang1,color="black",line_width=3)

GlyphRenderer(id='1795', ...)

In [77]:
s = bpl.figure(plot_width=400,plot_height=400,title="Distribución objetivo",toolbar_location=None)

In [78]:
s.line(x=x,y=y)

GlyphRenderer(id='1835', ...)

In [79]:
bpl.show(bly.row(p,s))

In [80]:
interp2 = sci.stats.gamma.ppf(interp1,a=a,scale=escala)

In [81]:
pd.Series(interp2,index=serie_data.index)

idTema  nombre_T                                                                         
871     repetido12                                                                             0.024252
5634    Capturan a sujeto tras robo a casa habitación en MH                                    0.024252
5332    CS Convocatoria para escultura por programa Sí al desarme Sí a la paz                  0.024252
8128    tema repetido Col. Florida                                                             0.024252
2697    Plaza Artz victimas eran israelíes y presuntos prófugos                                0.024252
                                                                                                ...    
12934   Choque de dos trenes en Metro Tacubaya deja como saldo 1 muerto y 41 heridos          90.075149
827     Localizan el cuerpo de Norberto Ronquillo en Xochimilco                               90.702642
12400   Confirman primer caso de coronavirus en CDMX                          

In [82]:
frame["inter2"] = interp2

In [83]:
frame[frame["inter2"]>100]

,,Alcance_suma,inter1,inter2
idTema,nombre_T,,,
11945,FGJ dio a conocer retrato hablado de mujer que raptó a Fátima,2.678091e+07,0.996954,100.0


In [84]:
frame.describe()

,Alcance_suma,inter1,inter2
count,9.984000e+03,9984.000000,9984.000000
mean,3.057873e+05,0.499909,13.819849
std,1.064801e+06,0.288565,15.833088
min,5.711978e+01,0.010000,0.024252
25%,1.173326e+04,0.250000,2.595299
50%,4.410650e+04,0.499992,8.148226
75%,1.734337e+05,0.750000,19.230126
max,2.678091e+07,0.996954,100.000000


In [85]:
k = bpl.figure(plot_width=400,plot_height=400,title="Distribución acumulativa empírica reescalada")

In [86]:
k.circle(x=interp2,y=interp1,color="blue",radius=0.7,alpha=0.1)
k.line(x=x,y=y,color="black")

GlyphRenderer(id='2045', ...)

In [87]:
bpl.show(k)

In [88]:
frame.loc[19695]

,Alcance_suma,inter1,inter2
nombre_T,,,
Hieren a mujer a balazos dentro de su auto en col. Buenos Aires Cuauhtémoc,1762.534041,0.06807,0.379507


In [89]:
hist = np.histogram(interp2,bins=100)
hist2 = np.histogram(serie_data,bins=100)

In [90]:
p = bpl.figure(title="Distribución de alcances, " + ciudad,toolbar_location=None,plot_width=400,plot_height=400,)
s = bpl.figure(title="Distribución de calificaciones reescalada, " + ciudad,toolbar_location=None,plot_width=400,plot_height=400,)

In [91]:
s.vbar(x=hist[1],top=hist[0],width=(hist[1][1]-hist[1][0])*0.9)
p.vbar(x=hist2[1],top=hist2[0],width=(hist2[1][1]-hist2[1][0])*0.9)

GlyphRenderer(id='2249', ...)

In [92]:
bpl.show(bly.row(p,s))